In [1]:
# 한글 폰트 설정
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

print("한글 폰트 설치 완료!")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (6,354 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 126374 files and dire

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 1. 기존 리포지토리 폴더로 이동
import os
os.chdir('/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project')

In [3]:
# 2. 경로 확인
!pwd

/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project


In [4]:
# 3. 최신 변경사항 가져오기
!git pull origin main

From https://github.com/Dongjin-1203/Codeit_AI_4th_Drug_image_CV_project
 * branch            main       -> FETCH_HEAD
Already up to date.


In [5]:
# 3. 현재 상태 확인
!git status

Refresh index: 100% (13/13), done.
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   notebooks/create_dataset.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data_preprocess/Untitled0.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


# 데이터 전처리 코드

## 초기 설정

In [19]:
import os
import json
import glob
from PIL import Image
import pandas as pd
from collections import defaultdict, Counter

## 1.🔍 이미지 품질검사

In [6]:
def check_image_quality(data_path="./data"):
    """이미지 품질 검사"""
    print("🔍 이미지 품질 검사 시작...")

    # 결과 저장용
    results = {
        'total_images': 0,
        'valid_images': 0,
        'corrupted_images': [],
        'small_images': [],
        'invalid_format': [],
        'image_stats': []
    }

    # 훈련 이미지 검사
    train_images = glob.glob(os.path.join(data_path, "train_images", "*.png"))
    test_images = glob.glob(os.path.join(data_path, "test_images", "*.png"))
    all_images = train_images + test_images

    results['total_images'] = len(all_images)
    print(f"📊 총 이미지 수: {len(all_images)}개")

    for i, img_path in enumerate(all_images):
        try:
            # 이미지 열기 시도
            with Image.open(img_path) as img:
                width, height = img.size
                mode = img.mode

                # 최소 해상도 검사 (100x100)
                if width < 100 or height < 100:
                    results['small_images'].append({
                        'path': img_path,
                        'size': (width, height)
                    })
                    continue

                # RGB 채널 확인
                if mode not in ['RGB', 'L']:  # L은 grayscale
                    results['invalid_format'].append({
                        'path': img_path,
                        'mode': mode
                    })
                    continue

                # 정상 이미지
                results['valid_images'] += 1
                results['image_stats'].append({
                    'path': img_path,
                    'width': width,
                    'height': height,
                    'mode': mode,
                    'size_mb': os.path.getsize(img_path) / (1024*1024)
                })

        except Exception as e:
            # 손상된 이미지
            results['corrupted_images'].append({
                'path': img_path,
                'error': str(e)
            })

        # 진행률 출력
        if (i + 1) % 100 == 0:
            print(f"  진행률: {i + 1}/{len(all_images)}")

    # 결과 요약
    print(f"\n📋 이미지 품질 검사 결과:")
    print(f"  ✅ 정상 이미지: {results['valid_images']}개")
    print(f"  ❌ 손상된 이미지: {len(results['corrupted_images'])}개")
    print(f"  ⚠️ 너무 작은 이미지: {len(results['small_images'])}개")
    print(f"  🔄 잘못된 형식: {len(results['invalid_format'])}개")

    return results

## 2. 🔍 어노테이션 무결성 검사

In [10]:
def check_annotation_integrity(data_path="./data"):
    """어노테이션 무결성 검사"""
    print("\n🔍 어노테이션 무결성 검사 시작...")

    results = {
        'total_annotations': 0,
        'valid_annotations': 0,
        'corrupted_json': [],
        'missing_pairs': [],
        'invalid_bbox': [],
        'annotation_stats': []
    }

    # 훈련 이미지와 어노테이션 매칭 확인
    train_images = glob.glob(os.path.join(data_path, "train_images", "*.png"))

    for img_path in train_images:
        img_name = os.path.basename(img_path).replace('.png', '')

        # 해당하는 JSON 파일 찾기
        json_files = glob.glob(os.path.join(data_path, "train_annotations", "*", "*", f"{img_name}.json"))

        if not json_files:
            # 매칭되는 어노테이션이 없음
            results['missing_pairs'].append({
                'image': img_path,
                'reason': 'No matching JSON file'
            })
            continue

        json_path = json_files[0]
        results['total_annotations'] += 1

        try:
            # JSON 파일 읽기 시도
            with open(json_path, 'r', encoding='utf-8') as f:
                annotation_data = json.load(f)

            # 이미지 크기 가져오기 (바운딩 박스 검증용)
            try:
                with Image.open(img_path) as img:
                    img_width, img_height = img.size
            except:
                continue

            # 바운딩 박스 검증 (기본적인 검사)
            bbox_valid = True
            if 'annotations' in annotation_data:
                for ann in annotation_data['annotations']:
                    if 'bbox' in ann:
                        bbox = ann['bbox']
                        # bbox 형식 확인 [x, y, width, height] 또는 [x_min, y_min, x_max, y_max]
                        if len(bbox) == 4:
                            x, y, w, h = bbox
                            # 좌표가 이미지 범위 내에 있는지 확인
                            if x < 0 or y < 0 or x + w > img_width or y + h > img_height:
                                bbox_valid = False
                                break

            if not bbox_valid:
                results['invalid_bbox'].append({
                    'annotation': json_path,
                    'image_size': (img_width, img_height),
                    'reason': 'Bbox outside image boundaries'
                })
                continue

            # 정상 어노테이션
            results['valid_annotations'] += 1

            # 통계 정보 수집
            pill_codes = []
            if 'annotations' in annotation_data:
                for ann in annotation_data['annotations']:
                    if 'category_id' in ann:
                        pill_codes.append(ann['category_id'])

            results['annotation_stats'].append({
                'path': json_path,
                'image_path': img_path,
                'num_objects': len(annotation_data.get('annotations', [])),
                'pill_codes': pill_codes
            })

        except json.JSONDecodeError as e:
            # JSON 파싱 오류
            results['corrupted_json'].append({
                'path': json_path,
                'error': str(e)
            })
        except Exception as e:
            # 기타 오류
            results['corrupted_json'].append({
                'path': json_path,
                'error': str(e)
            })

    # 결과 요약
    print(f"📋 어노테이션 무결성 검사 결과:")
    print(f"  ✅ 정상 어노테이션: {results['valid_annotations']}개")
    print(f"  ❌ 손상된 JSON: {len(results['corrupted_json'])}개")
    print(f"  🔗 매칭되지 않는 쌍: {len(results['missing_pairs'])}개")
    print(f"  📐 잘못된 바운딩 박스: {len(results['invalid_bbox'])}개")

    return results

## 3.📊 데이터셋 기본 통계 정보 생성

In [14]:
def generate_dataset_statistics(image_results, annotation_results):
    """데이터셋 기본 통계 정보 생성"""
    print("\n📊 데이터셋 통계 정보 생성...")

    # 이미지 통계
    if image_results['image_stats']:
        widths = [stat['width'] for stat in image_results['image_stats']]
        heights = [stat['height'] for stat in image_results['image_stats']]
        sizes_mb = [stat['size_mb'] for stat in image_results['image_stats']]

        print(f"\n📐 이미지 크기 통계:")
        print(f"  평균 크기: {sum(widths)/len(widths):.0f} x {sum(heights)/len(heights):.0f}")
        print(f"  최소 크기: {min(widths)} x {min(heights)}")
        print(f"  최대 크기: {max(widths)} x {max(heights)}")
        print(f"  평균 파일 크기: {sum(sizes_mb)/len(sizes_mb):.2f} MB")

    # 어노테이션 통계
    if annotation_results['annotation_stats']:
        total_objects = sum(stat['num_objects'] for stat in annotation_results['annotation_stats'])
        avg_objects_per_image = total_objects / len(annotation_results['annotation_stats'])

        print(f"\n🏷️ 어노테이션 통계:")
        print(f"  총 객체 수: {total_objects}개")
        print(f"  이미지당 평균 객체 수: {avg_objects_per_image:.1f}개")

        # 알약 코드 분포 (간단하게)
        all_codes = []
        for stat in annotation_results['annotation_stats']:
            all_codes.extend(stat['pill_codes'])

        if all_codes:
            code_counts = Counter(all_codes)
            print(f"  고유 알약 종류: {len(code_counts)}개")
            print(f"  가장 많은 알약 코드:")
            for code, count in code_counts.most_common(5):
                print(f"    {code}: {count}개")

## 4. 📋 품질 검사 보고서 생성

In [15]:
def create_quality_report(image_results, annotation_results, output_path="./data"):
    """품질 검사 보고서 생성"""
    print(f"\n📋 품질 검사 보고서 생성 중...")

    report = {
        'summary': {
            'total_images': image_results['total_images'],
            'valid_images': image_results['valid_images'],
            'total_annotations': annotation_results['total_annotations'],
            'valid_annotations': annotation_results['valid_annotations'],
            'overall_quality_score': (
                (image_results['valid_images'] / max(image_results['total_images'], 1) * 0.5) +
                (annotation_results['valid_annotations'] / max(annotation_results['total_annotations'], 1) * 0.5)
            ) * 100
        },
        'issues': {
            'corrupted_images': len(image_results['corrupted_images']),
            'small_images': len(image_results['small_images']),
            'corrupted_json': len(annotation_results['corrupted_json']),
            'missing_pairs': len(annotation_results['missing_pairs']),
            'invalid_bbox': len(annotation_results['invalid_bbox'])
        }
    }

    # JSON 형태로 저장
    report_path = os.path.join(output_path, "quality_report.json")
    with open(report_path, 'w', encoding='utf-8') as f:
        json.dump(report, f, indent=2, ensure_ascii=False)

    print(f"✅ 보고서 저장 완료: {report_path}")
    print(f"\n🎯 전체 품질 점수: {report['summary']['overall_quality_score']:.1f}%")

    return report

## 5. 🔍 발견된 문제들 상세 출력

In [16]:
def print_detailed_issues(image_results, annotation_results, max_display=5):
    """발견된 문제들 상세 출력"""
    print(f"\n🔍 발견된 문제들 (최대 {max_display}개씩 표시):")

    # 손상된 이미지
    if image_results['corrupted_images']:
        print(f"\n❌ 손상된 이미지 ({len(image_results['corrupted_images'])}개):")
        for i, issue in enumerate(image_results['corrupted_images'][:max_display]):
            print(f"  {i+1}. {issue['path']}")
            print(f"     오류: {issue['error']}")

    # 너무 작은 이미지
    if image_results['small_images']:
        print(f"\n⚠️ 너무 작은 이미지 ({len(image_results['small_images'])}개):")
        for i, issue in enumerate(image_results['small_images'][:max_display]):
            print(f"  {i+1}. {issue['path']}")
            print(f"     크기: {issue['size']}")

    # 손상된 JSON
    if annotation_results['corrupted_json']:
        print(f"\n❌ 손상된 어노테이션 ({len(annotation_results['corrupted_json'])}개):")
        for i, issue in enumerate(annotation_results['corrupted_json'][:max_display]):
            print(f"  {i+1}. {issue['path']}")
            print(f"     오류: {issue['error']}")

    # 매칭되지 않는 쌍
    if annotation_results['missing_pairs']:
        print(f"\n🔗 매칭되지 않는 이미지-어노테이션 쌍 ({len(annotation_results['missing_pairs'])}개):")
        for i, issue in enumerate(annotation_results['missing_pairs'][:max_display]):
            print(f"  {i+1}. {issue['image']}")

## 6. 🚀 전체 품질 검사 실행

In [17]:
def run_quality_check(data_path="./data"):
    """전체 품질 검사 실행"""
    print("🚀 데이터 품질 검사 시작!")
    print("=" * 50)

    # 1. 이미지 품질 검사
    image_results = check_image_quality(data_path)

    # 2. 어노테이션 무결성 검사
    annotation_results = check_annotation_integrity(data_path)

    # 3. 통계 정보 생성
    generate_dataset_statistics(image_results, annotation_results)

    # 4. 품질 보고서 생성
    report = create_quality_report(image_results, annotation_results, data_path)

    # 5. 상세 문제 출력
    print_detailed_issues(image_results, annotation_results)

    print("\n" + "=" * 50)
    print("✅ 데이터 품질 검사 완료!")

    return image_results, annotation_results, report

## 메인 실행부

In [20]:
if __name__ == "__main__":
    # 코랩에서 실행
    print("📖 사용법:")
    print("run_quality_check('./data')  # 전체 품질 검사 실행")
    print("run_quality_check('./small_data')  # 소규모 데이터셋 검사")

    # 파일 경로들
    prototype_path = "./data/prototype_data"
    data_path = "./data"

    # 함수 실행
    # image_results, annotation_results, report = run_quality_check(data_path)    # 전체 데이터용
    image_results, annotation_results, report = run_quality_check(prototype_path)   # 소규모 데이터용

📖 사용법:
run_quality_check('./data')  # 전체 품질 검사 실행
run_quality_check('./small_data')  # 소규모 데이터셋 검사
🚀 데이터 품질 검사 시작!
🔍 이미지 품질 검사 시작...
📊 총 이미지 수: 75개

📋 이미지 품질 검사 결과:
  ✅ 정상 이미지: 75개
  ❌ 손상된 이미지: 0개
  ⚠️ 너무 작은 이미지: 0개
  🔄 잘못된 형식: 0개

🔍 어노테이션 무결성 검사 시작...
📋 어노테이션 무결성 검사 결과:
  ✅ 정상 어노테이션: 50개
  ❌ 손상된 JSON: 0개
  🔗 매칭되지 않는 쌍: 0개
  📐 잘못된 바운딩 박스: 0개

📊 데이터셋 통계 정보 생성...

📐 이미지 크기 통계:
  평균 크기: 976 x 1280
  최소 크기: 976 x 1280
  최대 크기: 976 x 1280
  평균 파일 크기: 1.73 MB

🏷️ 어노테이션 통계:
  총 객체 수: 50개
  이미지당 평균 객체 수: 1.0개
  고유 알약 종류: 14개
  가장 많은 알약 코드:
    3482: 14개
    1899: 9개
    3350: 7개
    3543: 6개
    2482: 5개

📋 품질 검사 보고서 생성 중...
✅ 보고서 저장 완료: ./data/prototype_data/quality_report.json

🎯 전체 품질 점수: 100.0%

🔍 발견된 문제들 (최대 5개씩 표시):

✅ 데이터 품질 검사 완료!


In [ ]:
# 전역 설정 (권장) - 이 Colab 세션에서 계속 사용
!git config --global user.name "Dongjin-1203"
!git config --global user.email "hambur1203@gmail.com"

In [ ]:
!git add .

In [ ]:
!git commit -m "EDA: 이미지 심층 분석, JSON 심층 분석 추가. sample 데이터 출력 추가. 마지막 EDA 보고서 초안 작성"

In [ ]:
!git pull origin main

In [ ]:
!git push origin main